# 1차 해결
- 1차 리뷰, 이름, 점수 가져오기
- 2차 다른 요소들()
- 3차 페이지 왔다갔다 구현

# 리뷰 크롤링 
- url 만 넣으면 가능
- 리뷰만 가지고옴

In [46]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import re
from datetime import datetime, timedelta

def convert_date(date_str):
    today = datetime.now()
    match_ago = re.search(r"(\d+) (week|day)s? ago", date_str)
    if match_ago:
        number, unit = match_ago.groups()
        number = int(number)
        if 'week' in unit:
            date = today - timedelta(weeks=number)
        elif 'day' in unit:
            date = today - timedelta(days=number)
        return date.strftime("%Y-%m-%d")
    match_month_year = re.search(r"(\w+) \d{4}", date_str)
    if match_month_year:
        return datetime.strptime(match_month_year.group(), "%B %Y").strftime("%Y-%m-%d")
    return "없음"

def airbnb_reviews(url):
    random_sec = random.uniform(1, 5)
    chrome_driver_path = 'C:/chromedriver-win64/chromedriver.exe'
    chrome_service = Service(chrome_driver_path)
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=chrome_service, options=options)
    driver.set_window_size(1920, 1080)
    driver.get(url)
    time.sleep(random_sec)

    try:
        translation_modal_close_button = WebDriverWait(driver, random_sec).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Close"]'))
        )
        translation_modal_close_button.click()
        time.sleep(1)
    except Exception as e:
        print("번역 모달 창 실패 :", e)

    try:
        review_button = WebDriverWait(driver, random_sec).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show all') and contains(text(), 'reviews')]"))
        )
        review_button.click()
    except Exception as e:
        print("리뷰 버튼 실패 :", e)
        driver.quit()
        return []

    try:
        scroll_panel = WebDriverWait(driver, random_sec).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="pdp-reviews-modal-scrollable-panel"]'))
        )
    except Exception as e:
        print("리뷰 모달창 실패 :", e)
        driver.quit()
        return []

    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scroll_panel)
    time.sleep(1)

  # 무한 스크롤을 통해 모든 리뷰를 로드
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)

    for _ in range(2):
        # 스크롤을 내리는 동작 (패널의 마지막 부분으로 이동)
        driver.execute_script("arguments[0].scrollIntoView(false);", scroll_panel)
        time.sleep(1)  # 스크롤 후 로딩을 위해 잠시 대기

        # 새로운 스크롤 높이를 가져오기
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)
        last_height = new_height  # 다음 반복을 위해 마지막 높이를 업데이트

    reviews = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_container = soup.find('div', {'data-testid': 'pdp-reviews-modal-scrollable-panel'})
    if review_container:
        review_tags = review_container.find_all('span', class_='lrl13de atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr')
        for i in range(1, len(review_tags) + 1):
            text = review_tags[i-1].text.strip()
            date_elements = driver.find_elements(By.XPATH, f"/html/body/div[9]/div/div/section/div/div/div[2]/div/div[3]/div/div/div/section/div/section/div/div[2]/div[2]/div/div[{i}]/div[1]/div")
            if date_elements:
                dates = [element.text for element in date_elements if element.text.strip() != '']
                dates = [convert_date(date) for date in dates]
                if dates:  
                    reviews.append({'리뷰': text, '리뷰날짜': dates[0]})
                else:
                    reviews.append({'리뷰': text, '리뷰날짜': '날짜 정보 없음'})
            else:
                reviews.append({'리뷰': text, '리뷰날짜': '날짜 정보 없음'})
    driver.quit()

    reviews_df = pd.DataFrame(reviews)
    reviews_df['리뷰날짜'] = pd.to_datetime(reviews_df['리뷰날짜'],errors='coerce', format="%Y-%m-%d")
    filtered_reviews_df = reviews_df[(reviews_df['리뷰날짜'] >= "2024-04-01") & (reviews_df['리뷰날짜'] <= datetime.now())]

    return filtered_reviews_df


df_subset = df[['숙소_id', 'url']]

# 새로운 DataFrame 생성
paris_reviews = pd.DataFrame(columns=['url', '숙소_id', '리뷰', '리뷰날짜'])

# 각 숙소에 대해 리뷰 정보 수집
for index, row in df_subset.iterrows():
    review_data = airbnb_reviews(row['url'])
    if index == 2:
        break
    review_data['url'] = row['url']
    review_data['숙소_id'] = row['숙소_id']
    paris_reviews = pd.concat([paris_reviews, review_data], ignore_index=True)



NameError: name 'df' is not defined